In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re

In [3]:
url = """https://insights.blackcoffer.com/environmental-impact-of-the-covid-19-pandemic-lesson-for-the-future/"""

In [4]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
title


'Environmental impact of the COVID-19 pandemic – Lesson for the Future'

In [5]:
content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
content

' The Covid- 19 pandemics forced factories to shut down, flights getting canceled and a massive decrease in the global economy, with a significant decrease in Green House Gases (GHG) in many developed and developing countries. The SARS- CoV2 came into the spotlight in December 2019 and has impacted most of the countries till then. Nearly 131 million peoples were infected worldwide and resulting in deaths of around 2.9 Million according to World Health Organisation (WHO). Most of the countries dealt with the new virus by imposing strict lockdowns and social distancing to control the spread of the virus. These policies caused adverse effects worldwide. One of the most important impacts of the Covid-19 Pandemic is on the environment. There have been few positive impacts on the environment due to lockdown like, air pollution has decreased dramatically, as people were asked to stay in their houses due to the lockdowns. There has also been a sharp decline in environmental noise. Environmenta

In [6]:
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
content

' The Covid 19 pandemics forced factories to shut down flights getting canceled and a massive decrease in the global economy with a significant decrease in Green House Gases GHG in many developed and developing countries The SARS CoV2 came into the spotlight in December 2019 and has impacted most of the countries till then Nearly 131 million peoples were infected worldwide and resulting in deaths of around 29 Million according to World Health Organisation WHO Most of the countries dealt with the new virus by imposing strict lockdowns and social distancing to control the spread of the virus These policies caused adverse effects worldwide One of the most important impacts of the Covid19 Pandemic is on the environment There have been few positive impacts on the environment due to lockdown like air pollution has decreased dramatically as people were asked to stay in their houses due to the lockdowns There has also been a sharp decline in environmental noise Environmental noise can be well 

In [7]:
text = content.split()
text


['The',
 'Covid',
 '19',
 'pandemics',
 'forced',
 'factories',
 'to',
 'shut',
 'down',
 'flights',
 'getting',
 'canceled',
 'and',
 'a',
 'massive',
 'decrease',
 'in',
 'the',
 'global',
 'economy',
 'with',
 'a',
 'significant',
 'decrease',
 'in',
 'Green',
 'House',
 'Gases',
 'GHG',
 'in',
 'many',
 'developed',
 'and',
 'developing',
 'countries',
 'The',
 'SARS',
 'CoV2',
 'came',
 'into',
 'the',
 'spotlight',
 'in',
 'December',
 '2019',
 'and',
 'has',
 'impacted',
 'most',
 'of',
 'the',
 'countries',
 'till',
 'then',
 'Nearly',
 '131',
 'million',
 'peoples',
 'were',
 'infected',
 'worldwide',
 'and',
 'resulting',
 'in',
 'deaths',
 'of',
 'around',
 '29',
 'Million',
 'according',
 'to',
 'World',
 'Health',
 'Organisation',
 'WHO',
 'Most',
 'of',
 'the',
 'countries',
 'dealt',
 'with',
 'the',
 'new',
 'virus',
 'by',
 'imposing',
 'strict',
 'lockdowns',
 'and',
 'social',
 'distancing',
 'to',
 'control',
 'the',
 'spread',
 'of',
 'the',
 'virus',
 'These',
 'p

In [8]:
len(text)

1375

In [9]:
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

35


In [10]:
#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)

66


In [11]:
filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

# Adding Subjectivity & Polarity
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)

data

,url,title,content,filter_content,Positive_Score,Negative_Score,polarity,subjectivity
0,https://insights.blackcoffer.com/environmental...,Environmental impact of the COVID-19 pandemic ...,The Covid 19 pandemics forced factories to sh...,The Covid 19 pandemics forced factories to shu...,35,66,0.044159,0.395886


In [12]:
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=print(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)


Word average = 7032.0
FOG INDEX =  553.59
Average no of words per sentence
1375.0
Complex Words 2512


In [13]:
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
print(syllable_count/len(content.split()))

Word Count 8413
Percentage of Complex Words 29.858552240580057
Average Word per Length 5.114181818181819
The AVG number of syllables in the word is: 
2.0647272727272727


In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
url = """https://insights.blackcoffer.com/environmental-impact-of-the-covid-19-pandemic-lesson-for-the-future/"""
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
# title

content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
# print(content)
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
# print(content)
text = content.split()
# print(text)
len(text)
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)


filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
print(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
syllable = (syllable_count/len(content.split()))
print(syllable)

data = [[url,title,content,filter_content,Positive_score,Negative_score,AVG_SENTENCE_LENGTH,pcw,FOG_INDEX,
         AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score","Avg_Sentence_Length"
                               ,"Percentage_Complex_Word","Fog_Index"," AVG_NUMBER_OF_WORDS_PER_SENTENCE","COMPLEX_WORDS",
                               "Word_Count","syllable","Average_Word_Length"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data



35
66
Word average = 7032.0
FOG INDEX =  553.59
Average no of words per sentence
1375.0
Complex Words 2512
Word Count 8413
Percentage of Complex Words 29.858552240580057
Average Word per Length 5.114181818181819
The AVG number of syllables in the word is: 
2.0647272727272727


,url,title,content,filter_content,Positive_Score,Negative_Score,Avg_Sentence_Length,Percentage_Complex_Word,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length,polarity,subjectivity
0,https://insights.blackcoffer.com/environmental...,Environmental impact of the COVID-19 pandemic ...,The Covid 19 pandemics forced factories to sh...,The Covid 19 pandemics forced factories to shu...,35,66,7032.0,29.858552,553.59,1375.0,2512,8413,2.064727,5.114182,0.044159,0.395886


In [2]:
data.to_csv(r'C:\Users\Om Bhandwalkar\Desktop\BlackCoffer Assignment\Output\url_58.csv')